
# Import Data

Importing AXP data.

In [63]:

import pandas as pd

forex_df = pd.read_csv('../../data/gen/AUDUSD_Daily_df.csv',
                       parse_dates=True,
                       index_col=0)
# Strip whitespace from column names
forex_df.columns = forex_df.columns.str.strip()
# Print
forex_df.head()


,timestamp,open,high,low,close,pre_close,p_change,pip_change
,,,,,,,,
4999,2001-05-10,0.5233,0.5267,0.5217,0.5237,0.5233,0.000764,7.643799
4998,2001-05-11,0.5237,0.5252,0.5184,0.5216,0.5237,-0.004010,-40.099293
4997,2001-05-14,0.5213,0.5224,0.5154,0.5193,0.5216,-0.004410,-44.095092
4996,2001-05-15,0.5193,0.5212,0.5150,0.5206,0.5193,0.002503,25.033699
4995,2001-05-16,0.5209,0.5240,0.5182,0.5233,0.5206,0.005186,51.863235



# Even Faster Approach: Using Numba

Optimize Calculation using Numba

## Processing Data

Adding slope & velocity to each day.


In [64]:

import numpy as np

extra_df = forex_df.copy()
extra_df['slope'] = pd.Series(np.gradient(extra_df.close), extra_df.index, name='slope')
extra_df['velocity'] = pd.Series(np.gradient(extra_df.slope), extra_df.index, name='velocity')
# Print
extra_df.head()


,timestamp,open,high,low,close,pre_close,p_change,pip_change,slope,velocity
,,,,,,,,,,
4999,2001-05-10,0.5233,0.5267,0.5217,0.5237,0.5233,0.000764,7.643799,-0.00210,-0.000100
4998,2001-05-11,0.5237,0.5252,0.5184,0.5216,0.5237,-0.004010,-40.099293,-0.00220,0.000800
4997,2001-05-14,0.5213,0.5224,0.5154,0.5193,0.5216,-0.004410,-44.095092,-0.00050,0.002100
4996,2001-05-15,0.5193,0.5212,0.5150,0.5206,0.5193,0.002503,25.033699,0.00200,0.002075
4995,2001-05-16,0.5209,0.5240,0.5182,0.5233,0.5206,0.005186,51.863235,0.00365,0.000450



## Performing Backtests

Assuming trading with Leverage

1. Prepare Parameters
1. Dfine a DataFrame to handle results
1. For Loops
1. Prepare Maps & Filters

## Preparing Parameters


In [65]:

import itertools

buy_slope_threshold_list = np.arange(-0.1, 0.1, 0.05)
buy_velocity_threshold_list = np.arange(-0.5, 0.0, 0.05)
close_slope_threshold_list = np.arange(-0.2, 0.2, 0.05)
close_velocity_threshold_list = np.arange(-0.2, 0.2, 0.05)
sell_slope_threshold_list = np.arange(-0.1, 0.1, 0.05)
sell_velocity_threshold_list = np.arange(0.0, 0.5, 0.05)

task_list = list(itertools.product(
    buy_slope_threshold_list, 
    buy_velocity_threshold_list, 
    close_slope_threshold_list, 
    close_velocity_threshold_list, 
    sell_slope_threshold_list, 
    sell_velocity_threshold_list
))
print("Backtest Params Ready, {} tasks to run. Will take about {} minutes to complete.".format(len(task_list), round(len(task_list)/(375*60), 2)))


Backtest Params Ready, 102400 tasks to run. Will take about 4.55 minutes to complete.



## Maths for Forex Margin Trading

### Without Leverage: 

Equity required to trade a standard lot of AUDUSD would be **AUD$100,000** converted to USD.

Assuming **1 pip rise** in price, aka 0.01%, aka 0.0001 change in price, will result calculate as $100,000 * 0.01% profit. **Profit = AUD$10 - SWAPs**.

Or simply **1 pip = $10 of the base currency**

Therefore, Return is AUD$10 / AUD$100,000 = **0.01%** (Or simply, 1 pip). 

**Return Fomula without Leverage** is therefore:

> Return = 1 pip * 1 = 0.01% = 0.0001 (p_change * 1)

The 1 means 1:1 Leverage, or no leverage.

### With Leverage:

Let's assume the following:

Leverage: 400

Equity required to trade a standard lot of AUDUSD would be AUD$100,000 / 400, or **AUD$250** converted to USD.

Assuming **1 pip rise** in price, aka 0.01%, aka 0.0001 change in price, will result calculate as $100,000 * 0.01% profit. **Profit = AUD$10 - SWAPs**.

Or simply **1 pip = $10 of the base currency**

Therefore, **Return Fomula with Leverage** is AUD$10 / AUD$250 = **4%** (or simply, 400 pip).

**Return Fomula without Leverage** is therefore:

> Return = 1 pip * 400 = 4% = 0.04 (p_change * 400)

The 400 means 1:400 Leverage. 



## Backtesting with 1:400 Leverage


In [ ]:

import concurrent.futures
from concurrent.futures.process import ProcessPoolExecutor
from datetime import datetime
from tqdm import tqdm

margin_interest = 0.1 # Assuming Margin interest is 10% per year
leverage = 400 ## Assuming Trading with Leverage

backtest_df = extra_df[:-504] # We use all available data until 2 years ago
    
# Defining a DataFrame to handle results
backtest_result = pd.DataFrame(columns=['buy_slope_threshold', 'buy_velocity_threshold', 'close_slope_threshold', 'close_velocity_threshold', 'sell_slope_threshold', 'sell_velocity_threshold', 'profit_percentage'])

def single_test(data_df, 
                buy_slope_threshold, 
                buy_velocity_threshold, 
                close_slope_threshold, 
                close_velocity_threshold, 
                sell_slope_threshold, 
                sell_velocity_threshold):
        
        # Prepare Maps & Filters 
        
        long_mask = (data_df.slope > buy_slope_threshold) & (data_df.velocity > buy_velocity_threshold)
        long_close = (data_df.slope < -close_slope_threshold) & (data_df.velocity < -close_velocity_threshold)
        long_series = data_df.p_change[long_mask]
        long_array = long_series.values * leverage # Assuming Forex Margin Trading,
        margin_cost = margin_interest / 365 * len(long_series) 
        long_profit = np.product(long_array/100 + 1) - 1 - margin_cost
        
        short_mask = (data_df.slope < sell_slope_threshold) & (data_df.velocity < sell_velocity_threshold)
        short_close = (data_df.slope > close_slope_threshold) & (data_df.velocity > close_velocity_threshold)
        short_series = data_df.p_change[short_mask]
        short_array = short_series.values * leverage # Assuming Forex Margin Trading,
        margin_cost = margin_interest / 365 * len(short_array)
        short_profit = np.abs(np.product(short_array/100 - 1)) - 1 - margin_cost
    
        trade_profit = long_profit + short_profit
    
        test_result = [buy_slope_threshold, 
                       buy_velocity_threshold, 
                       close_slope_threshold, 
                       close_velocity_threshold, 
                       sell_slope_threshold, 
                       sell_velocity_threshold, 
                       trade_profit * 100]
        return test_result

start_time = datetime.now()

with ProcessPoolExecutor() as executor:
    """
    By default, ProcessPool uses maximum available number of cores to process.
    """
    
    """
    Equivalent to executor.map(fn, *iterables),
    but displays a tqdm-based progress bar.
    
    Does not support timeout or chunksize as executor.submit is used internally
    
    **kwargs is passed to tqdm.
    """
    futures_list = []
    kwargs = {
        'total': len(futures_list),
        'mininterval': 30.0,
        'unit': 'tests',
        'unit_scale': True,
        'leave': True
    }
        
    futures_list = [executor.submit(single_test, 
                                    backtest_df,
                                    buy_slope_threshold, 
                                    buy_velocity_threshold, 
                                    close_slope_threshold, 
                                    close_velocity_threshold, 
                                    sell_slope_threshold, 
                                    sell_velocity_threshold) for buy_slope_threshold, \
                                                                 buy_velocity_threshold, \
                                                                 close_slope_threshold, \
                                                                 close_velocity_threshold, \
                                                                 sell_slope_threshold, \
                                                                 sell_velocity_threshold in task_list]
    print("Running {} tasks".format(len(futures_list)))
    
    for f in tqdm(concurrent.futures.as_completed(futures_list), **kwargs):
        # done_callback runs on the main process
        result_row = f.result()
        
        backtest_result = backtest_result.append(pd.Series(result_row, index=backtest_result.columns), ignore_index=True)

    print("Finished {} tasks in {}".format(len(task_list), datetime.now() - start_time))


0.00tests [00:00, ?tests/s]

Running 102400 tasks



## Analysing Backtest Results


In [ ]:
        
backtest_result = backtest_result.sort_values(by=['profit_percentage'],
                                        ascending=False)
backtest_result.head(100)


In [ ]:

margin_interest = 0.1 # Assuming Margin interest is 10% per year
leverage = 400 # Assuming Trading with Leverage

test_df = extra_df[-504:] # We use all available data until 2 years ago
test_params = backtest_result.iloc[:100]

test_list = list(zip(test_params.buy_slope_threshold, 
                    test_params.buy_velocity_threshold,
                    test_params.close_slope_threshold, 
                    test_params.close_velocity_threshold,
                    test_params.sell_slope_threshold, 
                    test_params.sell_velocity_threshold,))

print("Params Ready, {} tasks to run. ".format(len(test_list)))
    

In [61]:
# Defining a DataFrame to handle results
test_result = pd.DataFrame(columns=['buy_slope_threshold', 'buy_velocity_threshold', 'close_slope_threshold', 'close_velocity_threshold', 'sell_slope_threshold', 'sell_velocity_threshold', 'profit_percentage'])

def single_test(data_df, 
                buy_slope_threshold, 
                buy_velocity_threshold, 
                close_slope_threshold, 
                close_velocity_threshold, 
                sell_slope_threshold, 
                sell_velocity_threshold):
        
        # Prepare Maps & Filters 
        
        long_mask = (data_df.slope > buy_slope_threshold) & (data_df.velocity > buy_velocity_threshold)
        long_close = (data_df.slope < -close_slope_threshold) & (data_df.velocity < -close_velocity_threshold)
        long_series = data_df.p_change[long_mask]
        long_array = long_series.values * leverage # Assuming Forex Margin Trading,
        margin_cost = margin_interest / 365 * len(long_series)
        long_profit = np.product(long_array/100 + 1) - 1 - margin_cost
        
        short_mask = (data_df.slope < sell_slope_threshold) & (data_df.velocity < sell_velocity_threshold)
        short_close = (data_df.slope > close_slope_threshold) & (data_df.velocity > close_velocity_threshold)
        short_series = data_df.p_change[short_mask]
        short_array = short_series.values * leverage # Assuming Forex Margin Trading,
        margin_cost = margin_interest / 365 * len(short_array)
        short_profit = np.abs(np.product(short_array/100 - 1)) - 1 - margin_cost
    
        trade_profit = long_profit + short_profit
    
        test_result = [buy_slope_threshold, 
                       buy_velocity_threshold, 
                       close_slope_threshold, 
                       close_velocity_threshold, 
                       sell_slope_threshold, 
                       sell_velocity_threshold, 
                       trade_profit * 100]
        return test_result

start_time = datetime.now()

with ProcessPoolExecutor() as executor:
    """
    By default, ProcessPool uses maximum available number of cores to process.
    """
    
    """
    Equivalent to executor.map(fn, *iterables),
    but displays a tqdm-based progress bar.
    
    Does not support timeout or chunksize as executor.submit is used internally
    
    **kwargs is passed to tqdm.
    """
    futures_list = []
    kwargs = {
        'total': len(futures_list),
        'mininterval': 30.0,
        'unit': 'tests',
        'unit_scale': True,
        'leave': True
    }
        
    futures_list = [executor.submit(single_test, 
                                    test_df,
                                    buy_slope_threshold, 
                                    buy_velocity_threshold, 
                                    close_slope_threshold, 
                                    close_velocity_threshold, 
                                    sell_slope_threshold, 
                                    sell_velocity_threshold) for buy_slope_threshold, \
                                                                 buy_velocity_threshold, \
                                                                 close_slope_threshold, \
                                                                 close_velocity_threshold, \
                                                                 sell_slope_threshold, \
                                                                 sell_velocity_threshold in test_list]
    print("Running {} tests".format(len(futures_list)))
    
    for f in tqdm(concurrent.futures.as_completed(futures_list), **kwargs):
        # done_callback runs on the main process
        result_row = f.result()
        
        test_result = test_result.append(pd.Series(result_row, index=test_result.columns), ignore_index=True)

    print("Finished {} tests in {}".format(len(test_list), datetime.now() - start_time))


100tests [00:00, 491tests/s]


Running 100 tests
Finished 100 tests in 0:00:00.256364



## Analysing Forward Test Results

In [62]:
        
test_result = test_result.sort_values(by=['profit_percentage'],
                                        ascending=False)
test_result.head()


,buy_slope_threshold,buy_velocity_threshold,close_slope_threshold,close_velocity_threshold,sell_slope_threshold,sell_velocity_threshold,profit_percentage
0,0.0,-0.3,1.500000e-01,0.15,0.0,0.45,2102.457773
63,0.0,-0.2,-5.551115e-17,0.15,0.0,0.10,2102.457773
73,0.0,-0.2,-5.000000e-02,0.05,0.0,0.05,2102.457773
72,0.0,-0.2,-5.000000e-02,-0.05,0.0,0.25,2102.457773
71,0.0,-0.2,-5.000000e-02,-0.05,0.0,0.20,2102.457773
